In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [2]:
from tqdm import tqdm
from pathlib import Path
from src.utils.config_loader import load_config
from src.utils.seed import seed_everything

base_dir = Path(os.getcwd()).parent

config = load_config(base_dir / 'secrets.yaml')

seed_everything(42)

In [3]:
from src.data.preprocessing import create_df

val_df = create_df(base_dir / 'data/my_data/regplans-dev.conllu')

In [4]:
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import (SystemMessage, HumanMessage)

os.environ['OPENAI_API_VERSION'] = config['OPENAI_API_VERSION']
os.environ['AZURE_OPENAI_ENDPOINT'] = config['OPENAI_API_BASE']
os.environ['AZURE_OPENAI_API_KEY'] = config['OPENAI_API_KEY']

llm = AzureChatOpenAI(
    deployment_name=config['OPENAI_DEPLOYMENT_NAME'],
    temperature=0.0
)

In [5]:
import json
import random

def format_examples(example_subset): 
    # Formats the examples into a string for later prompt
    formatted = []
    for i, ex in enumerate(example_subset):
        entity_lines = "\n".join([f"{e['word']} {e['label']}" for e in ex["entities"]])
        formatted.append(f"Eksempel {i+1}:\nTekst: \"{ex['sentence']}\"\nEntiteter:\n{entity_lines}\n##\n")
    
    return "\n".join(formatted)

with open(base_dir / 'llm_stuff/prompts/examples.json', 'r') as f:
    example_bank = json.load(f)

ids = [5, 19, 1, 21, 16]

#ids = random.sample(range(1, 26), 3)

examples = [next(ex for ex in example_bank if ex["id"] == id) for id in ids]

formatted_examples = format_examples(examples)

print(formatted_examples)

Eksempel 1:
Tekst: "2.02 Bygningenepå feltene BBB1 , BFS 1 og BFS2 skal plasseres innenfor byggegrensene som er vist på plankartet ."
Entiteter:
BBB1 B-FELT
BFS B-FELT
1 I-FELT
BFS2 B-FELT
##

Eksempel 2:
Tekst: "Parkeringsplassar ( SPP ) Grøntstruktur , jf . PBL § 12-5 , 2 . ledd nr . 3 - Turveg ( GT )"
Entiteter:
SPP B-FELT
GT B-FELT
##

Eksempel 3:
Tekst: "Adkomst til BFS1 og BFS2 skal være fra Solfjellveien ."
Entiteter:
BFS1 B-FELT
BFS2 B-FELT
##

Eksempel 4:
Tekst: "Areal brattere enn 1 : 3 , arealer i gul eller rød sone for henholdsvis støy ( T-1442 ) og luftkvalitet ( T-1520 ) ."
Entiteter:

##

Eksempel 5:
Tekst: "Før det vert gjeve mellombels bruksløyve / ferdigattest for ny bueining innanfor felt BKS1 og BFS14 og 15"
Entiteter:
BKS1 B-FELT
BFS14 B-FELT
og I-FELT
15 I-FELT
##



In [6]:
from src.utils.label_mapping_regplans import label_to_id
from collections import defaultdict

all_pred_ids = []
all_true_ids = []
all_results = []

# val_df = val_df.iloc[:int(len(val_df) * 0.5)] # Use only half the data for testing

for idx, row in tqdm(val_df.iterrows(), total=len(val_df)):

    sentence = row['full_text']
    tokens = row['words']
    true_labels = row['labels']  

    msg = [
        SystemMessage(
            content=(
                "Du er en ekspert på Named Entity Recognition (NER). Din oppgave er å identifisere entiteter "
                "som representerer feltnavn i tekstutdrag fra reguleringsplaner."
            )
        ),
        HumanMessage(
            content=f"""\
    De eneste gyldige etikettene er B-FELT (begynnelsen på et feltnavn) og I-FELT (fortsettelsen av det samme feltnavnet).

    {formatted_examples}

    Formuler svaret over flere linjer, med ett token per linje, og kun tokens som inngår i ett feltnavn. Hver linje skal inneholde tokenet etterfulgt av tilhørende etikett, atskilt med ett mellomrom.

    Tekst: '{sentence}'

    Entiteter:
    """
        )
    ]


    try:
        response = llm.invoke(msg)

        entities = defaultdict(list) # Word-label pairs

        for line in response.content.splitlines():
            parts = line.strip().split()
            if len(parts) == 2:
                word, label = parts[0], parts[1]
                entities[word].append(label)

        pred_labels = []
        word_counts = defaultdict(int)  # Track occurrences of each word

        for token in tokens:
            if token in entities and word_counts[token] < len(entities[token]):
                pred_labels.append(entities[token][word_counts[token]])  # Get the label in order
                word_counts[token] += 1  # Increment occurrence counter
            else:
                pred_labels.append("O")  # Default to "O" if missing

        # Convert labels to IDs
        pred_ids = []
        for label in pred_labels:
            if label in label_to_id:
                pred_ids.append(label_to_id[label])
            else:
                pred_ids.append(label_to_id.get("O", -1))

        true_ids = [label_to_id[label] for label in true_labels]

        all_pred_ids.extend(pred_ids)
        all_true_ids.extend(true_ids)

        all_results.append({
            'sentence': sentence,
            'tokens': tokens,
            'true_labels': true_labels,
            'predicted_labels': pred_labels,
            'generated_text': response.content
        })   
        
    except Exception as e:
        print(f"Skipping row {idx} due to error: {e}")
        continue   

  0%|          | 0/353 [00:00<?, ?it/s]

100%|██████████| 353/353 [35:25<00:00,  6.02s/it]  


In [7]:
from llm_stuff.evaluation import evaluate 

metrics = evaluate(all_true_ids, all_pred_ids)

print("Evaluation Metrics on Val Set:")
print(metrics)

final_output = {
    'prompt': str(msg),
    'evaluation_metrics': metrics,
    'results': all_results
}

with open(base_dir / f"llm_stuff/results/{config['OPENAI_DEPLOYMENT_NAME']}_FEWSHOT_5_Norwegian_FULLVALSET_BEST_SUBSET_T2.json", 'w', encoding='utf-8') as f:
    json.dump(final_output, f, indent=4, ensure_ascii=False)

Evaluation Metrics on Val Set:
{'precision': 0.7488901615142822, 'recall': 0.6613999009132385, 'f1': 0.695984959602356, 'span_acc': 0.751937985420227, 'classification_report': {'B-FELT': {'precision': 0.8859649122807017, 'recall': 0.7829457364341085, 'f1-score': 0.831275720164609, 'support': 129.0}, 'I-FELT': {'precision': 0.375, 'recall': 0.20689655172413793, 'f1-score': 0.26666666666666666, 'support': 29.0}, 'O': {'precision': 0.9857054070851461, 'recall': 0.9943573667711598, 'f1-score': 0.9900124843945068, 'support': 3190.0}, 'accuracy': 0.9793906810035843, 'macro avg': {'precision': 0.7488901064552825, 'recall': 0.6613998849764687, 'f1-score': 0.6959849570752609, 'support': 3348.0}, 'weighted avg': {'precision': 0.976572497695886, 'recall': 0.9793906810035843, 'f1-score': 0.9776307426681734, 'support': 3348.0}}}
